In [ ]:
# import math
import random
from random import shuffle
import numpy as np

class Domino:
    
    def buy(self,hand, n, pieces):
        i = 0
        while i<n:
            hand.append(pieces[i])
            i=i+1
        del pieces[0:n]
        return hand

    #recebe estado do jogo, retorna ações possíveis
    def possibleActions(self,state,player):
        status = state[0]
        if(player =="p1"):
            hand = state[1]
        else:
            hand = state[2]
        l_end = state[4]
        r_end = state[5]
        actions = []
        index = -1

        if(l_end==r_end==-1):
            for piece in hand:
                index +=1
                actions.append((piece[0],piece[1],index))
            return actions

        for piece in hand:
            index +=1
            #peça dupla
            if(piece[0]==piece[1]):
                if(piece[0]==l_end):
                    actions.append((piece[1],"left",index))
                elif(piece[1]==r_end):
                    actions.append((piece[0],"right",index))
                continue
            if(piece[0]==l_end):
                actions.append((piece[1],"left",index))
                if(l_end==r_end): #evitar duplicacao de mesmas ações na esq e dir
                    continue
            if(piece[1]==l_end):
                actions.append((piece[0],"left",index))
                if(l_end==r_end):
                    continue
            if(piece[0]==r_end):
                actions.append((piece[1],"right",index))
                if(l_end==r_end):
                    continue
            if(piece[1]==r_end):
                actions.append((piece[0],"right",index))
                if(l_end==r_end):
                    continue
        '''
        pos 2 e 3 no actions se referem a nova ponta na mesa:
        exemplo: campo 1-5 , mão : (1,2),(5,5)
        nesse caso a peça (1,2) se encaixa no lado esquerdo, sendo
        a nova ponta esquerda o 2. A representação será: (0,1), pois
        0 é a posição da tupla na mão, e o 1 representa a posição na tupla,
        que no caso é o direito

        '''
        if not actions:
            actions.append(None)
        return actions

    def termination(self,state):#se a partida ja terminou
        status = state[0]
        p1_hand = state[1]
        p2_hand = state[2]
        l_end = state[4]
        r_end = state[5]
        if (status >= 3):
            return True  
        return False

    def reward(self,state,player):#recompensa por estar nesse estado
        #em caso de vitoria
        status = state[0]

        if(status == 1 or status == 2):
            return 0
        if(status == 3):
            p1_hand = state[1]
            p2_hand = state[2]
            lighestP1Value = 13
            lighestP2Value = 13
            lighestP1Tile = 13
            lighestP2Tile = 13
            p1_total_value = 0
            p2_total_value = 0
            for piece in p1_hand:
                value = piece[0]+piece[1]
                p1_total_value+=value
            for piece in p2_hand:
                value = piece[0]+piece[1]
                p2_total_value+=value
            if(p1_total_value<p2_total_value):
                state[0] = 4#player 1 venceu tendo a menor mao
                if(player == "p1"):
                    return 1
                else:
                    return -1
            elif(p1_total_value>p2_total_value):
                state[0] = 5#player 2 venceu tendo a menor mao
                if(player == "p1"):
                    return -1
                else:
                    return 1
            else:#empate
                for piece in p1_hand:
                    if(piece[0]<lighestP1Value):
                        lighestP1Value = piece[0]
                        lighestP1Tile = piece[0]+piece[1]
                    if(piece[1]<lighestP1Value):
                        lighestP1Value = piece[1]
                        lighestP1Tile = piece[0]+piece[1]
                        
                for piece in p2_hand:
                    if(piece[0]<lighestP2Value):
                        lighestP2Value = piece[0]
                        lighestP2Tile = piece[0]+piece[1]
                    if(piece[1]<lighestP2Value):
                        lighestP2Value = piece[1]
                        lighestP2Tile = piece[0]+piece[1]
                        
                if(lighestP1Value<lighestP2Value):
                    state[0] = 4#player 1 venceu tendo o menor valor entre o par das pecas    
                    if(player == "p1"):
                        return 1
                    else:
                        return -1
                elif(lighestP1Value>lighestP2Value):
                    state[0] = 5#player 2 venceu tendo o menor valor entre o par das pecas
                    if(player == "p1"):
                        return -1
                    else:
                        return 1
                else:
                    if(lighestP1Tile<lighestP2Tile):#menor valor da peça como um todo
                        state[0] = 4
                        if(player == "p1"):
                            return 1
                        else:
                            return -1
                    else:#nao pode haver empate
                        state[0] = 5
                        if(player == "p1"):
                            return -1
                        else:
                            return 1

    def startGame(self):
        status = 1 #1=in progress; 2=one player blocked;3=two players blocked;4/5=p1 won/p2 won
        pieces = [(x,y) for x in range(7) for y in range(x,7)]
        random.shuffle(pieces)
        p1_hand = self.buy([],7,pieces)
        p2_hand = self.buy([],7,pieces)
        field = []
        l_end = -1
        r_end = -1
        state = [status, p1_hand, p2_hand, field, l_end, r_end]
        return state

    def playGame(self,state,action,player):
        status = state[0]
        if(status >= 3): #jogo acabou
            return state
        if(player =="p1"):
            hand = state[1]
        else:
            hand = state[2]
            
        field = state[3]
        l_end = state[4]
        r_end = state[5]

        if(action[0] is None):#foi bloqueado
            if(status == 2):
                state[0] = 3
            elif(status == 1):
                state[0] = 2
            return state
        
        if(status == 2): # o oponente foi bloqueado, mas eu tenho peça
            state[0] = 1
        p_index = action[2]  
        if(l_end==r_end==-1):
            orientation = "left"
        else:
            orientation = action[1]
        p = hand[p_index]
        field.append(p)
        hand.remove(p)
        if (l_end==-1 and r_end==-1):
            l_end, r_end = p
        elif (orientation == "right"):#ori e o lado desejado a manter na ponta
            r_end=action[0]
        elif (orientation == "left"):
            l_end=action[0]
                
        if(player =="p1"):
            state[1] = hand
        else:
            state[2] = hand
            
        if(len(hand) ==0): #zerou a mao, acabou o jogo
            state[0] = 3
        state[3] = field
        state[4] = l_end
        state[5] = r_end
        return state



In [73]:

class Features:
    
    def __init__(self,beta,featureWeights):
        self.data = []
        self.beta = beta
        self.featureWeights = featureWeights
        self.numFeatures = featureWeights.size
        
    def getFeaturesWeights(self):
        return self.featureWeights
    
    def setFeaturesWeights(self,featureWeights):
        self.featureWeights = featureWeights
    
    def featureValues(self,state,action,player):
        result =  np.zeros(self.numFeatures)
        l_end = state[4]
        r_end = state[5]
        if(player == "p1"):
            hand = state[1]
            handOp = state[2]
        elif(player == "p2"):
            hand = state[2]
            handOp = state[1]
        if(action[0]!=None):#quando nao for bloqueado    
            pos = action[2]
            if(l_end==r_end==-1):
                pos = action[2]
                l_end = action[0]
                r_end = action[1]
            elif(action[1]=="left"):
            #substitui uma das pontas pela nova ação
                l_end = action[0]
            else:
                r_end = action[0]
            #print "pos: "+str(pos)
            value =hand.pop(pos)
        """
        Características a partir de informações imperfeitas.
            Somente do que é visível a um jogador comum:
                Campo, peças de suas mãos, quantidade de peças do oponente
        """
        #valida caracteristicas de acordo com o estado submetido e as retorna
        
        result[0] = self.NumDouble(hand,action,l_end,r_end,lambda x: x==1)
        result[1] = self.NumDouble(hand,action,l_end,r_end,lambda x: x==2)
        result[2] = self.NumDouble(hand,action,l_end,r_end,lambda x: x>=3)
        #impossibilita qualquer peça de ser jogada
        result[3] = self.blocks(hand,action,l_end,r_end)
        
        result[4] = self.NumActions(hand,l_end,r_end,lambda x: x==1)
        result[5] = self.NumActions(hand,l_end,r_end,lambda x: x==2)
        result[6] = self.NumActions(hand,l_end,r_end,lambda x: x>=3)
       
        for i in range(7,14):
            #quais peças duplas temos na mão, de 0:0 a 6:6
            result[i] = self.hasDouble(i-7,hand)
                
        """
        Características a partir de informações perfeitas.
        Sabe-se da mão do oponente, de toda e qualquer informação necessária
        para se tomar uma decisão
        """
        result[14] = self.blocks(handOp,action,l_end,r_end)
        result[15] = self.NumActions(handOp,l_end,r_end,lambda x: x==1)
        result[16] = self.NumActions(handOp,l_end,r_end,lambda x: x==2)
        result[17] = self.NumActions(handOp,l_end,r_end,lambda x: x>=3)
        for i in range(18,25):
            #quais peças duplas temos na mão, de 0:0 a 6:6
            result[i] = self.shortPiece(i-18,handOp)
        
        if(self.numFeatures>25):
            result[25] = self.hasPieceValue(hand,lambda x: x>=9)
            result[26] = self.hasPieceValue(hand,lambda x: x<=3)
            result[27] = self.hasHandValue(hand,lambda x: x<=4)
            result[28] = self.hasHandValue(hand,lambda x: x>=12 and x<=19)
            result[29] = self.hasHandValue(hand,lambda x: x>=20)
            
            result[30] = self.hasPieceValue(handOp,lambda x: x>=9)
            result[31] = self.hasPieceValue(handOp,lambda x: x<=3)
            result[32] = self.hasHandValue(handOp,lambda x: x<=4)
            result[33] = self.hasHandValue(handOp,lambda x: x>=12 and x<=19)
            result[34] = self.hasHandValue(handOp,lambda x: x>=20)
        
        
        if(action[0]!=None):
            hand.insert(pos,value)
        
        return result
    
    
    def NumDouble(self,hand,action,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(piece[0] == piece[1]):
                count +=1
        if(exp(count)):
            return True
        return False
    
    def hasDouble(self,number,hand):
        for piece in hand:
            if((piece[0] == piece[1]) and (piece[0] == number)):
                return True
        return False
    
    def hasPieceValue(self,hand,exp):
        for piece in hand:
            value = piece[0] + piece[1]
            if(exp(value)):
                return True
        return False
    
    def hasHandValue(self,hand,exp):
        value = 0
        for piece in hand:
            value += piece[0] + piece[1]
        if(exp(value)):
            return True
        else:
            return False
    
    def blocks(self,hand,action,l_end,r_end):
        if(action[0] is None):
            return True
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                #se encontra alguma peça valida na mao
                return False # então ainda não estou bloqueado
  
        return True #nao encontrou
        
    def NumActions(self,hand,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                count +=1
        if(exp(count)):#se tem mais que o numero de peças designado
            return True
        return False #se tem exatamente aquele numero
    
    def shortPiece(self,number,handOp):
        for piece in handOp:
            if(piece[0]==number or piece[1]==number):
                return False
        return True #op n tem nenhuma peça dese tipo
        
          
    def isPlayable(self,piece,l_end,r_end):#se dado a peça e as duas pontas, ela é jogavel
        if(piece[0]==l_end or piece[0]==r_end or piece[1]==l_end or piece[1]==r_end):
            return True
        return False
    
    #Geracao de caracteristicas por combinacao de pares And
    def generateAndFeatures(self,state,action,player):
        result3 =  np.zeros(630)
        count = 0
        result1 = self.featureValues(state,action,player)
        for i in range(0,result1.size):
             for j in range (i,result1.size):
                    if(result1[i]==1 and result1[j]==1):
                        result3[count] = 1
                        count+=1
        
       

In [95]:
class Control:
    def __init__(self,player,features):
        self.player = player
        self.features = features
        self.featuresVector = np.zeros(35)
    #dado o jogador, sua acao, o estado e suas caracteristicas, retorne o valor dessa acao
    #e necessario diferencia os jogadores pois ele precisa examinar 2 hands 
    #o act n especifica de que hand e
    def getFeatures(self):
        return self.features
    
    def getPlayer(self):
        return self.player
    
    def generateVector(self,state,act):
        self.featuresVector = self.features.featureValues(state,act,self.player)
        
    def qValue(self,state,act):
        featureVals = self.features.featureValues(state,act,self.player)
        val = np.dot (self.features.getFeaturesWeights(), featureVals)    # value before action
        return val

    def eGreedyPicker(self,actions,state):
        rand = np.random.random_sample()
        bestValue = 0 #tomar cuidado aqui
        cond = False
        if(rand > e):
            for act in actions:  
                value = self.qValue(state,act)
                if(not cond):
                    bestValue = value
                    choosenAct = act
                    cond = True
                if(value> bestValue):
                    bestValue = value
                    choosenAct = act
        else:
            choosenAct = random.choice(actions)

        return choosenAct


    def policyAct(self,state):
        actions = dominoes.possibleActions(state,self.player)
        #escolhe a partir dos valores na q-value com tecnica e-greedy
        if(actions[0] is None):
            return actions
        else:
            choosenAct = self.eGreedyPicker(actions,state)

        return choosenAct

    def step(self,state,act):
        state = dominoes.playGame(state,act,self.player)                  # do selected action
        return state

    def alpha(self,vector):#calcua taxa de aprendizado de acordo com quantidade de features atuais
        unique, counts = np.unique(vector, return_counts=True)
        a = 0.1/(counts[1]) #constante por numero total de 1
        return a
    
    def evaluate(self,state,val,act):
        #A recompensa é dada após a ação dos 2 players
        #não é recompensa pela ação de 1 player, mas dos dois
        r = dominoes.reward(state,self.player)
        featureWeights = self.features.getFeaturesWeights()
        next_action = self.policyAct(state)                 
        new_value = self.qValue(state,next_action)
        target = r+ 0.9 * new_value                     # gamma = 0.9
        delta = target - val                            # prediction error
        featureWeights += alpha(self.featuresVector) * delta *self.featuresVector
        self.features.setFeaturesWeights(featureWeights)
        elements = [state,new_value,next_action]
        return elements



In [98]:
dominoes = Domino()
e = 0.03  # epsilon-greedy proportion
featureWeights = np.zeros(35)
features1 = Features(1,featureWeights)
controlP1 = Control("p1",features1)
controlP2 = Control("p2",features1)
r = 0
duration = 0
#self-play
firstplay = 0
for i in range(0,1000):
    state = dominoes.startGame()
    #firstplay = random.randint(0,1)#2 jogadores compartilhando a msm rede neural,mas so 1 avaliando

    actP1 = controlP1.policyAct(state)  
    val1 = controlP1.qValue(state,actP1)
    flag = 0
    while(dominoes.termination(state) == False):
        if(firstplay==0):#salva vetor de carac antes de mudar
            controlP1.generateVector(state,actP1)
        else:
            controlP2.generateVector(state,actP2)
        state = controlP1.step(state,actP1)        
        if(flag==0):
            actP2 = controlP2.policyAct(state)
            flag = 1
        val2 = controlP2.qValue(state,actP2)
        state = controlP2.step(state,actP2)
        if(firstplay==0):
            state,val1,actP1 =controlP1.evaluate(state,val1,actP1)
            actP2 = controlP2.policyAct(state)
        else:
            state,val2,actP2 =controlP2.evaluate(state,val2,actP2)
            actP1 = controlP1.policyAct(state)
   
featureWeights = controlP1.getFeatures().getFeaturesWeights()
print "Resulting weights: "+str(featureWeights)
np.save('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNetAnd', featureWeights)

Resulting weights: [-0.11032178 -0.11761096 -0.04990421  0.01879649  0.19865025  0.24216539
  0.20578233  0.28079298 -0.01462935 -0.13267257 -0.1707254  -0.11204059
 -0.19125608 -0.14200985 -0.11387595 -0.13354416 -0.03670204  0.00718328
  0.36521528 -0.2290457   0.09799631 -0.13859358 -0.07003191 -0.03362747
 -0.15475436 -0.10363412 -0.0324457   1.0114289  -0.39749697 -0.45473607
  0.20954073 -0.19097426 -0.73693466  0.37793375  0.61501344]


In [ ]:
dominoes = Domino()
e = 0.0001  # aqui n ha aprendizagem
featureWeightsP1 = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet4.npy')
featureWeightsP2 = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet5.npy')
featuresP1 = Features(1,featureWeightsP1)
featuresP2 = Features(1,featureWeightsP2)

r = 0
duration = 0

P1Wins = 0
firstplay = 1
for i in range(0,10000):
    state = dominoes.startGame()
    #firstplay = random.randint(0,1)
    while(dominoes.termination(state) == False):
        if(firstplay==0):
            actP1 = policyAct(state,"p1",featuresP1)         # take action by a e-greedy policy
            state = step(state,"p1",actP1)
            actP2 = policyAct(state,"p2",featuresP2)
            state = step(state,"p2",actP2)
        else:
            actP2 = policyAct(state,"p2",featuresP2)         # take action by a e-greedy policy
            state = step(state,"p2",actP2)
            actP1 = policyAct(state,"p1",featuresP1)
            state = step(state,"p1",actP1)
        r = dominoes.reward(state,"p1")
    if(state[0]==4):
        P1Wins+=1


print "Resulting weights: "+str(featureWeightsP1)
#np.save('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet3', featureWeightsP1)
print "Player1 winnings: "+str(P1Wins)

In [ ]:
dominoes = Domino()
e = 0.03  # epsilon-greedy proportion
state = dominoes.startGame()
#print "Initial state:" +str(state)
actionss1 = dominoes.possibleActions(state,"p1")
actP1 = policyAct(state,"p1",features)         # take action by a e-greedy policy
val1 = qValue(state,actP1,features,"p1")
#for i in range(0,1): 
while(dominoes.termination(state) == False):
        featuresVector = features.featureValues(state,actP1,"p1")
        state = step(state,"p1",actP1)
        print "State after p1 play: "+str(state[4]) + ","+str(state[5])
        print "my hand: "+str(state[2])
        actionss2 = dominoes.possibleActions(state,"p2")
        if(actionss2[0] is None):
            actP2 = policyAct(state,"p2",features)
            state = step(state,"p2",actP2)
        else:
            actionPos=input("Choose Action: "+str(actionss2))
            state = step(state,"p2",actionss2[actionPos])
        state,val1,actP1,featureWeights =evaluate(state,"p1",val1,actP1,features,featureWeights,featuresVector)
        print "State after both plays: "+str(state[0])